In [2]:
# Imports
from mountainlab_pytools import mdaio
from mountainlab_pytools import mlproc as mlp
import os

In [3]:
if not os.path.exists('dataset'):
    os.mkdir('dataset')
if not os.path.exists('output'):
    os.mkdir('output')

In [4]:
# Synthesize a dataset
duration=600
M=4
samplerate=30000
mlp.runProcess(
    'ephys.synthesize_random_waveforms',
    {},
    {
        'geometry_out':'dataset/geom.csv',
        'waveforms_out':'dataset/waveforms_true.mda'
    },
    {
        'upsamplefac':13,
        'M':M,
        'average_peak_amplitude':100
    }
)
mlp.runProcess(
    'ephys.synthesize_random_firings',
    {},
    {
        'firings_out':'dataset/firings_true.mda'
    },
    {
        'duration':duration
    }
)
mlp.runProcess(
    'ephys.synthesize_timeseries',
    {
        'firings':'dataset/firings_true.mda',
        'waveforms':'dataset/waveforms_true.mda'
    },
    {
        'timeseries_out':'dataset/raw.mda.prv'
    },{
        'duration':duration,
        'waveform_upsamplefac':13,
        'noise_level':10
    })

RUNNING:::::: ml-run-process ephys.synthesize_random_waveforms --inputs --parameters average_peak_amplitude:100 M:4 upsamplefac:13 --outputs waveforms_out:dataset/waveforms_true.mda geometry_out:dataset/geom.csv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Initializing process ... ]
[ Running ... ] /home/magland/main-venv/bin/python3 /home/magland/.mountainlab/packages/ml_ephys/synthesis/synthesis.py ephys.synthesize_random_waveforms --waveforms_out=/home/magland/src/mountainlab-js/examples/spike_sorting/python_example/dataset/waveforms_true.mda --geometry_out=/home/magland/src/mountainlab-js/examples/spike_sorting/python_example/dataset/geom.csv --average_peak_amplitude=100 --M=4 --upsamplefac=13 --_tempdir=/tmp/mountainlab-tmp/tempdir_faa0fa64b3_7PLTys
Elapsed time for processor ephys.synthesize_random_waveforms: 0.249 sec
[ 

{'timeseries_out': 'dataset/raw.mda.prv'}

In [5]:
# Run this only if you are running jupyter lab locally and have ephys-viz installed:
os.system('ev-timeseries dataset/raw.mda --firings dataset/firings_true.mda')

0

In [6]:
# Preprocessing
freq_min=300
freq_max=6000
mlp.runProcess(
    'ephys.bandpass_filter',
    {
        'timeseries':'dataset/raw.mda.prv'
    },{
        'timeseries_out':'output/filt.mda.prv'
    },
    {
        'samplerate':samplerate,
        'freq_min':freq_min,
        'freq_max':freq_max
    }
)
mlp.runProcess(
    'ephys.whiten',
    {
        'timeseries':'output/filt.mda.prv'
    },
    {
        'timeseries_out':'output/pre.mda.prv'
    },
    {}
)

RUNNING:::::: ml-run-process ephys.bandpass_filter --inputs timeseries:dataset/raw.mda.prv --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:output/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /home/magland/src/mountainlab-js/examples/spike_sorting/python_example/dataset/raw.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Initializing process ... ]
[ Running ... ] /home/magland/main-venv/bin/python3 /home/magland/.mountainlab/packages/ml_ephys/preprocessing/preprocessing.py ephys.bandpass_filter --timeseries=/tmp/mountainlab-tmp/output_187a8f79e734d3406f65d02c333fc9d20ef9fe16_timeseries_out.mda --timeseries_out=/tmp/mountainlab-tmp/output_eabb7dc4ea2bd92bf0b23e9658e77b4215ad30cb_timeseries_out.mda --freq_max=6000 --freq_min=300 --samplerate=30000 --_tempdir=/tmp/mountainlab-tmp/tempdir_eabb7dc4ea_nbVmTC
Chunk 

{'timeseries_out': 'output/pre.mda.prv'}

In [7]:
# Sorting
pp={}
pp['detect_sign']=1
pp['adjacency_radius']=-1
pp['detect_threshold']=3
mlp.runProcess(
    'ms4alg.sort',
    {
        'timeseries':'output/pre.mda.prv',
        'geom':'dataset/geom.csv'
    },
    {
        'firings_out':'output/firings.mda'
    },
    pp
)

RUNNING:::::: ml-run-process ms4alg.sort --inputs timeseries:output/pre.mda.prv geom:dataset/geom.csv --parameters detect_sign:1 adjacency_radius:-1 detect_threshold:3 --outputs firings_out:output/firings.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /home/magland/src/mountainlab-js/examples/spike_sorting/python_example/output/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Initializing process ... ]
[ Running ... ] /home/magland/main-venv/bin/python3 /home/magland/.mountainlab/packages/ml_ms4alg/ms4alg_spec.py ms4alg.sort --timeseries=/tmp/mountainlab-tmp/output_dae9dbabed11d0c1567c222476bfda2ce0d75ad3_timeseries_out.mda --geom=/home/magland/src/mountainlab-js/examples/spike_sorting/python_example/dataset/geom.csv --firings_out=/home/magland/src/mountainlab-js/examples/spike_sorting/python_example/output/firings.mda --detect_sign=1 --adjacency_ra

{'firings_out': 'output/firings.mda'}

In [8]:
# Compute templates
templates=mlp.runProcess(
    'ephys.compute_templates',
    {
        'timeseries':'output/filt.mda.prv',
        'firings':'output/firings.mda'
    },
    {
        'templates_out':True
    },
    {
        'clip_size':150
    }
)['templates_out']

RUNNING:::::: ml-run-process ephys.compute_templates --inputs timeseries:output/filt.mda.prv firings:output/firings.mda --parameters clip_size:150 --outputs templates_out:/tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_templates_out_d50aede235ea30ae58a526bf1c7c4a65c2b8b7af.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /home/magland/src/mountainlab-js/examples/spike_sorting/python_example/output/filt.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Initializing process ... ]
[ Running ... ] /home/magland/main-venv/bin/python3 /home/magland/.mountainlab/packages/ml_ephys/basic/basic.py ephys.compute_templates --timeseries=/tmp/mountainlab-tmp/output_eabb7dc4ea2bd92bf0b23e9658e77b4215ad30cb_timeseries_out.mda --firings=/home/magland/src/mountainlab-js/examples/spike_sorting/python_example/output/firings.mda --templates_out=/tmp/mountainlab-tmp/outp

In [11]:
# Compute metrics
metrics1=mlp.runProcess(
    'ms3.cluster_metrics',
    {
        'timeseries':'output/pre.mda.prv',
        'firings':'output/firings.mda'
    },
    {
        'cluster_metrics_out':True
    },
    {
        'samplerate':samplerate
    }
)['cluster_metrics_out']
metrics2=mlp.runProcess(
    'ms3.isolation_metrics',
    {
        'timeseries':'output/pre.mda.prv',
        'firings':'output/firings.mda'
    },
    {
        'metrics_out':True
    },
    {}
)['metrics_out']
metrics=mlp.runProcess(
    'ms3.combine_cluster_metrics',
    {
        'metrics_list':[metrics1,metrics2]
    },
    {
        'metrics_out':True
    },
    {}
)['metrics_out']
A=mlp.realizeFile(metrics)
print(A)

RUNNING:::::: ml-run-process ms3.cluster_metrics --inputs timeseries:output/pre.mda.prv firings:output/firings.mda --parameters samplerate:30000 --outputs cluster_metrics_out:/tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_cluster_metrics_out_aba9be54db3166188692248dd3915bf8c603529b.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /home/magland/src/mountainlab-js/examples/spike_sorting/python_example/output/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ms3.cluster_metrics already completed. ]
[ Creating output prv for cluster_metrics_out ... ]
[ Done. ]
RUNNING:::::: ml-run-process ms3.isolation_metrics --inputs timeseries:output/pre.mda.prv firings:output/firings.mda --parameters --outputs metrics_out:/tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_metrics_out_e37e87d88c8baac58f323a065459ea5eaf15035d.prv
[ Getting processor spec... ]
[ Checking inputs and substi

Exception: Non-zero exit code for ms3.combine_cluster_metrics

In [25]:
# Run this only if you are running jupyter lab locally and have ephys-viz installed:
os.system('ev-templates {}'.format(templates))

0